# test 전처리한 파일 불러오기

In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import gc
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
#test_df = pd.read_csv(r'/content/drive/MyDrive/open/데이콘전체전처리_test.csv', encoding='utf-8-sig')
test_df = pd.read_csv(r'/content/drive/MyDrive/open/0_1_4_test.csv', encoding='utf-8-sig')

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 430 entries, 기준년월 to _2순위신용체크구분_체크
dtypes: bool(48), float64(12), int64(370)
memory usage: 1.7 GB


# train으로 진행한 모델 파일 불러오기

In [5]:
# 먼저 설치 필요 (Colab이나 Jupyter 환경에서는 아래 실행 필요)
!pip install xgboost catboost lightgbm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 1.3 MB/s eta 0:00:00


In [ ]:
from sklearn.linear_model import LogisticRegression

In [38]:
with open('/content/drive/MyDrive/open/voting_model(inf_상관관계1제거_D)','rb') as f: #voting, stacking, bagging,rf
    voting_dlf= pickle.load(f)

In [7]:
test_df

,기준년월,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,입회경과개월수_신용,유효카드수_신용체크,유효카드수_신용,유효카드수_신용_가족,유효카드수_체크,이용가능카드수_신용체크,...,직장시도명_울산,직장시도명_인천,직장시도명_전남,직장시도명_전북,직장시도명_제주,직장시도명_충남,직장시도명_충북,_2순위신용체크구분_신용,_2순위신용체크구분_없음,_2순위신용체크구분_체크
0,0,2,2,270,51,2,2,1,0,2,...,False,False,False,False,False,False,False,True,False,False
1,0,1,1,291,30,2,1,0,1,2,...,False,True,False,False,False,False,False,False,False,True
2,0,2,2,316,5,2,2,0,0,2,...,False,False,False,False,False,False,False,True,False,False
3,0,1,1,248,73,1,1,0,0,1,...,False,True,False,False,False,False,False,False,True,False
4,0,1,1,145,176,3,2,0,1,3,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,5,0,0,256,69,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
599996,5,1,1,311,14,1,1,0,0,1,...,False,False,False,False,False,False,False,False,True,False
599997,5,1,1,319,6,1,1,0,0,1,...,False,False,False,False,False,False,False,False,True,False
599998,5,3,3,243,82,6,4,0,2,6,...,False,False,False,False,False,False,False,True,False,False


# train에서 사용한 컬럼만 추출

In [ ]:
test_df = test_df.drop(columns=['ID'])

KeyError: "['ID'] not found in axis"

In [39]:
columns_to_drop=['보유여부_해외겸용_본인_1', '보유여부_해외겸용_신용_본인_1', '수신거부여부_DM_0',
       '수신거부여부_TM_1', '수신거부여부_메일_0', '이용가능여부_해외겸용_본인_1',
       '이용가능여부_해외겸용_신용_본인_0', '이용여부_3M_해외겸용_본인_1', '이용여부_3M_해외겸용_신용_본인_1',
       '청구서발송여부_B0_1', '청구서발송여부_R3M_1', '청구서발송여부_R6M_1']

# 실제 존재하는 컬럼만 필터링해서 드롭
columns_to_drop_actual = [col for col in columns_to_drop if col in test_df.columns]
test_df = test_df.drop(columns=columns_to_drop_actual)


In [ ]:
test_df.shape

(600000, 419)

In [40]:
#학습 때 사용한 컬럼들 중에서, 테스트 데이터에 존재하는 컬럼들만 추림
common_cols = [col for col in voting_dlf.feature_names_in_ if col in test_df.columns]#voting, stacking,bagging,rf
test_df = test_df[common_cols]
test_df

,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,입회경과개월수_신용,유효카드수_신용체크,유효카드수_신용,유효카드수_신용_가족,유효카드수_체크,이용가능카드수_신용체크,이용가능카드수_신용,...,직장시도명_울산,직장시도명_인천,직장시도명_전남,직장시도명_전북,직장시도명_제주,직장시도명_충남,직장시도명_충북,_2순위신용체크구분_신용,_2순위신용체크구분_없음,_2순위신용체크구분_체크
0,2,2,270,51,2,2,1,0,2,2,...,False,False,False,False,False,False,False,True,False,False
1,1,1,291,30,2,1,0,1,2,1,...,False,True,False,False,False,False,False,False,False,True
2,2,2,316,5,2,2,0,0,2,2,...,False,False,False,False,False,False,False,True,False,False
3,1,1,248,73,1,1,0,0,1,1,...,False,True,False,False,False,False,False,False,True,False
4,1,1,145,176,3,2,0,1,3,2,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0,0,256,69,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
599996,1,1,311,14,1,1,0,0,1,1,...,False,False,False,False,False,False,False,False,True,False
599997,1,1,319,6,1,1,0,0,1,1,...,False,False,False,False,False,False,False,False,True,False
599998,3,3,243,82,6,4,0,2,6,4,...,False,False,False,False,False,False,False,True,False,False


In [41]:
preds = voting_dlf.predict(test_df) #voting, stacking
test_df['Segment'] = preds

In [43]:
print(len(test_df))

600000


In [ ]:
# 1. 정확한 컬럼 순서로 정렬
X_test = test_df[model.feature_names_in_]

# 2. 예측 수행
preds = model.predict(X_test)
test_df['Segment'] = preds


In [ ]:
test_df.shape

(600000, 418)

In [11]:
# 라벨 인코딩에 대한 매핑 딕셔너리
label_mapping = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E'
}

In [42]:
# 'Segment' 컬럼의 값들을 원래 값으로 변환
test_df['Segment'] = test_df['Segment'].map(label_mapping)


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 418 entries, 소지카드수_유효_신용 to Segment
dtypes: bool(36), float64(12), int64(369), object(1)
memory usage: 1.7+ GB


# 데이콘 제출 파일 형식에 맞춰서 변환

In [33]:
# 100,000개의 ID 값을 생성
ids = [f"TEST_{str(i).zfill(5)}" for i in range(100000)]

# 6번 반복
repeated_ids = ids * 6

# test_df의 ID 컬럼에 넣기 (행 수가 맞는지 확인)
test_df['ID'] = repeated_ids[:len(test_df)]

In [ ]:
test_df.shape

(600000, 419)

In [34]:
submission = test_df.groupby("ID")["Segment"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]

In [35]:
submission.to_csv('/content/drive/MyDrive/open/test_df(softvoting_D_tuning_상관계수1제거).csv', index=False) #voting,stacking,bagging,rf